In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn
pd.set_option('use_inf_as_na', True)
from collections import Counter
import pickle

In [2]:
raw_data = pd.read_pickle(r'/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 2/HW1/dataset.pkl')
data = raw_data[raw_data['market_cap'] > 1000.0]
data = data.copy()
data.fillna(0.0,inplace=True)

In [3]:
def f(x):
    if x > 0.01:
        return 1
    elif x < -0.01:
        return -1
    else:
    
        return 0

In [4]:
data['rel_performance'] = data['pred_rel_return'].apply(f)
data.reset_index(inplace=True)
data.set_index('date',inplace=True)

In [5]:
df_1 = data.loc['2000-01-01':'2003-01-01']
df_valid = data.loc['2013-04-01':'2013-07-01']
df_test = data.loc['2013-07-01':'2013-10-01']

In [26]:
train_1 = df_1.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)

valid = df_valid.reset_index().drop(['ticker','date',
                                    'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)
test = df_test.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)

,actq,apq,atq,ceqq,cheq,cogsq,csh12q,cshfdq,cshiq,cshopq,...,sector_code_815.0,sector_code_817.0,sector_code_822.0,sector_code_823.0,sector_code_825.0,sector_code_830.0,sector_code_835.0,sector_code_840.0,sector_code_845.0,sector_code_850.0
0,7722.000,482.000,21391.000,16523.000,3968.000,1422.000,3374.1250,3648.000,3445.000,0.0,...,0,0,0,0,0,0,0,0,0,0
1,172.725,19.662,474.649,239.432,3.198,47.634,30.2688,31.217,0.000,0.0,...,0,0,0,0,0,0,0,0,0,0
2,240.767,27.044,376.536,209.411,68.625,43.023,21.4360,23.753,0.000,0.0,...,0,0,0,0,0,0,0,0,0,0
3,7681.000,3538.000,11471.000,5308.000,4132.000,5452.000,2536.0000,2731.000,2575.000,0.0,...,0,0,0,0,0,0,0,0,0,0
4,507.082,139.497,1094.080,402.382,27.605,221.366,43.1858,43.687,53.321,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10039,3293.843,897.120,6287.869,-483.813,236.962,2885.039,515.2413,515.124,515.115,0.0,...,0,0,0,0,0,0,0,0,0,0
10040,963.500,177.000,1857.200,751.900,8.100,281.800,38.6250,40.600,39.172,0.0,...,0,0,0,0,0,0,0,0,0,0
10041,6314.200,1472.100,10608.000,4721.400,1997.600,2435.400,822.8250,825.500,824.800,0.0,...,0,0,0,0,0,0,0,0,0,0
10042,1516.934,89.176,2969.655,667.721,931.795,715.809,134.1400,134.180,134.390,0.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_1_stock_returns = df_1['next_period_return']
valid_stock_returns = df_valid['next_period_return']
test_stock_returns = df_test['next_period_return']

y_1 = df_1['rel_performance']
y_valid = df_valid['rel_performance']
y_test = df_test['rel_performance']

y_1 = y_1.values
y_valid = y_valid.values
y_test = y_test.values

## Importing Optuna and suppressing warning messages

In [8]:
import optuna
from optuna.trial import Trial
optuna.logging.set_verbosity(optuna.logging.FATAL)
import warnings
warnings.filterwarnings("ignore")

## Defining the Optuna objective function

In [9]:
def objective(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):

    rf_n_estimators = trial.suggest_int('n_estimators', 10,40,step=5)
    rf_max_features = trial.suggest_categorical('max_features',['sqrt','log2'])
    rf_min_samples_leaf = trial.suggest_int('min_samples_leaf',800,2400,step=800)
    rf_max_depth = trial.suggest_int('max_depth',4,15)
    
    rf_clf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                    max_depth=rf_max_depth,
                                    min_samples_leaf=rf_min_samples_leaf,
                                    max_features=rf_max_features)
    rf_clf.fit(train,labels)
    accuracy = rf_clf.score(val,val_labels)
 
    return accuracy

In [10]:
study = optuna.create_study(direction="maximize")

In [11]:
from functools import partial

Use the included 3 sets of optimal features over the holding period 2003-2018 and a RandomForestClassifier. Use Optuna to find hyper-parameters for the training period 2000-01-01 to 2003-01-01 and use these hyper-parameters and the sets of optimal features to compute the maximum drawdown for each of them.

In [71]:
start_dates = [pd.to_datetime('2000-01-01') + pd.DateOffset(months = 3 * i) for i in range(62)]
end_dates = [d + pd.DateOffset(months = 36) for d in start_dates]

training_frames = [data.loc[d:d+pd.DateOffset(months = 36)] for d in start_dates]
validation_frames = [data.loc[d:d+pd.DateOffset(months = 3)] for d in end_dates]
test_frames = [data.loc[d + pd.DateOffset(months=3):d+pd.DateOffset(months = 6)] for d in end_dates]

training_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]
validation_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in validation_frames]
test_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]

training_labels = [d['rel_performance'].values for d in training_frames]
validation_labels = [d['rel_performance'].values for d in validation_frames]

In [55]:
with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 6/opt_feat_2.pkl','rb') as f:
    optimal_features_1 = pickle.load(f)
with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 6/opt_feat_3.pkl','rb') as f:
    optimal_features_2 = pickle.load(f)
with open('/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 6/opt_feat_4.pkl','rb') as f:
    optimal_features_3 = pickle.load(f)

[array([-1,  1,  1, ...,  1,  0,  1]),
 array([ 1, -1,  1, ...,  1,  1,  1]),
 array([-1,  1, -1, ..., -1,  1,  1]),
 array([ 1, -1,  1, ...,  1,  1,  1]),
 array([ 1,  1,  1, ...,  1,  1, -1]),
 array([ 1,  1, -1, ..., -1,  1,  1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([-1, -1, -1, ..., -1,  1,  1]),
 array([-1,  1,  1, ..., -1, -1, -1]),
 array([ 1, -1, -1, ...,  1,  1, -1]),
 array([-1,  1,  0, ...,  1, -1, -1]),
 array([-1, -1,  1, ...,  1,  0, -1]),
 array([-1,  1, -1, ...,  0,  1,  1]),
 array([ 1,  1,  1, ..., -1,  1, -1]),
 array([ 1,  0, -1, ..., -1,  1, -1]),
 array([ 1,  1,  1, ..., -1, -1,  1]),
 array([-1,  1,  1, ..., -1,  0, -1]),
 array([ 1, -1,  0, ...,  1,  1,  0]),
 array([-1, -1,  1, ...,  0, -1, -1]),
 array([ 1, -1,  1, ...,  1, -1,  1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([ 1, -1, -1, ...,  0,  1,  1]),
 array([-1, -1,  1, ...,  1,  1, -1]),
 array([ 1, -1,  0, ...,  0,  0,  1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([ 1, -1, -1, ...,  1, -1,  1]),
 arr

In [67]:
def Maxdrawdown(optimal_features):
    P_L = []
    x = [0,1]
    ret = []

    for i in range(1,len(optimal_features)-1):
            study = optuna.create_study(direction="maximize")
            study.optimize(partial(objective,train=training_data[i].loc[:,optimal_features[i-1]],labels=training_labels[i],val=validation_data[i].loc[:,optimal_features[i-1]],val_labels=validation_labels[i],val_rets=valid_stock_returns), n_trials=200,n_jobs=-1)
            rf_clf = RandomForestClassifier(**study.best_params)
            rf_clf.fit(training_data[i].loc[:,optimal_features[i-1]],training_labels[i])
            pred_i = rf_clf.predict(test_data[i][optimal_features[i-1]])

            profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
            P_L.append(profit_i)
            positions = np.sum(np.abs(pred_i))

            ret.append((1.0/positions) * profit_i)
            x.append(x[i] + (x[i]/positions) * profit_i)
            
            
            
    SPY = pd.read_pickle(r'/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 5/SPY_cum_ret.pkl')
    SPY = SPY.loc['2003-10-01':]
    SPY = SPY.resample('Q').ffill()
    SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
    SPY['strategy'] = x[1:]
    SPY = SPY.resample('A').bfill()

    returns = SPY['strategy']

    local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or 
       ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))] 

    local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
            (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]

    def next_local_min(n):
        if [m for m in local_min if m > n]:
            return [m for m in local_min if m > n][0]
        else: return None

    drawdowns = [(n,next_local_min(n)) for n in local_max]
    drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
    if drawdown_values:
        return  np.max(drawdown_values)
    else: return 0.0

In [68]:
Maxdrawdown(optimal_features_1)     ## Maxdrawdown using opt_feat_2

0.14549549248844262

In [69]:
Maxdrawdown(optimal_features_2) ## Maxdrawdown using opt_feat_3

0.3490585646126181

In [70]:
Maxdrawdown(optimal_features_3) ## Maxdrawdown using opt_feat_4

0.24455215827734555

Try different Optuna objectives such as accuracy and max drawdown on the period 2000-01-01 to 2003-04-01

In [74]:
start_dates = [pd.to_datetime('2000-01-01') + pd.DateOffset(months = 3 * i) for i in range(63)]
end_dates = [d + pd.DateOffset(months = 39) for d in start_dates]

training_frames = [data.loc[d:d+pd.DateOffset(months = 39)] for d in start_dates]
validation_frames = [data.loc[d:d+pd.DateOffset(months = 3)] for d in end_dates]
test_frames = [data.loc[d + pd.DateOffset(months=3):d+pd.DateOffset(months = 6)] for d in end_dates]

training_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]
validation_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in validation_frames]
test_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]

training_labels = [d['rel_performance'].values for d in training_frames]
validation_labels = [d['rel_performance'].values for d in validation_frames]


In [75]:
Maxdrawdown(optimal_features_1)   ###. period 2000-01-01 to 2003-04-01 maxdrawdown

0.6094104769274082

In [77]:
def sharpndrawdown(nestimatorstep, minsamplestep):
    def objective(trial:Trial,train=None,labels=None,val=None,val_labels=None,val_rets=None):

        rf_n_estimators = trial.suggest_int('n_estimators', 10,40,step=nestimatorstep)
        rf_max_features = trial.suggest_categorical('max_features',['sqrt','log2'])
        rf_min_samples_leaf = trial.suggest_int('min_samples_leaf',800,2400,step=minsamplestep)
        rf_max_depth = trial.suggest_int('max_depth',4,15)

        rf_clf = RandomForestClassifier(n_estimators=rf_n_estimators,
                                        max_depth=rf_max_depth,
                                        min_samples_leaf=rf_min_samples_leaf,
                                        max_features=rf_max_features)
        rf_clf.fit(train,labels)
        accuracy = rf_clf.score(val,val_labels)

        return accuracy

    start_dates = [pd.to_datetime('2000-01-01') + pd.DateOffset(months = 3 * i) for i in range(62)]
    end_dates = [d + pd.DateOffset(months = 36) for d in start_dates]

    training_frames = [data.loc[d:d+pd.DateOffset(months = 36)] for d in start_dates]
    validation_frames = [data.loc[d:d+pd.DateOffset(months = 3)] for d in end_dates]
    test_frames = [data.loc[d + pd.DateOffset(months=3):d+pd.DateOffset(months = 6)] for d in end_dates]

    training_data = [d.reset_index().drop
                                     (['ticker','date',
                                       'next_period_return',
                                       'spy_next_period_return',
                                       'rel_performance','pred_rel_return',
                                      'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]
    validation_data = [d.reset_index().drop(['ticker','date',
                                       'next_period_return',
                                       'spy_next_period_return',
                                       'rel_performance','pred_rel_return',
                                      'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in validation_frames]
    test_data = [d.reset_index().drop(['ticker','date',
                                       'next_period_return',
                                       'spy_next_period_return',
                                       'rel_performance','pred_rel_return',
                                      'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]

    training_labels = [d['rel_performance'].values for d in training_frames]
    validation_labels = [d['rel_performance'].values for d in validation_frames]    
    
    P_L = []
    x = [0,1]
    ret = []

    for i in range(1,len(optimal_features)-1):
            study = optuna.create_study(direction="maximize")
            study.optimize(partial(objective,train=training_data[i].loc[:,optimal_features[i-1]],labels=training_labels[i],val=validation_data[i].loc[:,optimal_features[i-1]],val_labels=validation_labels[i],val_rets=valid_stock_returns), n_trials=200,n_jobs=-1)
            rf_clf = RandomForestClassifier(**study.best_params)
            rf_clf.fit(training_data[i].loc[:,optimal_features[i-1]],training_labels[i])
            pred_i = rf_clf.predict(test_data[i][optimal_features[i-1]])

            profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
            P_L.append(profit_i)
            positions = np.sum(np.abs(pred_i))

            ret.append((1.0/positions) * profit_i)
            x.append(x[i] + (x[i]/positions) * profit_i)
            
            
    SPY = pd.read_pickle(r'/Users/peterfeng/Downloads/UChicago 2022 Winter/Machine Learning/Week 5/SPY_cum_ret.pkl')
    SPY = SPY.loc['2003-10-01':]
    SPY = SPY.resample('Q').ffill()
    SPY['spy_cum_ret'] = (SPY['spy_cum_ret'] - SPY['spy_cum_ret'][0] + 1)
    SPY['strategy'] = x[1:]
    SPY = SPY.resample('A').bfill()

    returns = SPY['strategy']
    strategy_mean_ret = (SPY['strategy'] - 1).diff().mean()
    strategy_std = (SPY['strategy'] - 1).diff().std()
    strategy_sr = strategy_mean_ret/strategy_std

    local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or 
       ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))] 

    local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
            (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]

    def next_local_min(n):
        if [m for m in local_min if m > n]:
            return [m for m in local_min if m > n][0]
        else: return None

    drawdowns = [(n,next_local_min(n)) for n in local_max]
    drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
    if drawdown_values:
        strategy_mdd = np.max(drawdown_values)
    else: strategy_mdd = 0.0
    
    result = "sharp ratio is: {}. max drawdown is: {}".format(strategy_sr,strategy_mdd)
    
    return result


In [78]:
sharpndrawdown(10, 10)

'sharp ratio is: 0.915512366349778. max drawdown is: 0.30572003763150724'

In [79]:
sharpndrawdown(20, 10)   ### nestimatorstep = 20, minsamplestep = 10 seems to have a better answer

'sharp ratio is: 1.0838751928161139. max drawdown is: 0.1966158326784515'

In [ ]:
sharpndrawdown(10, 20)